In [1]:
import numpy as np
import pandas as pd

In [2]:
# 한글이 포함된 csv파일을 읽어들이기 위해 encoding='cp949' 지정
raking = pd.read_csv('raking.csv', encoding='cp949') 
raking

,성별,전공,w
0,1,2,1
1,2,1,1
2,1,2,1
3,1,1,1
4,2,2,1
...,...,...,...
95,1,2,1
96,1,2,1
97,2,1,1
98,1,1,1


In [3]:
# Raking전 '성별'의 분포 (단위: %)
raking.groupby('성별')['w'].sum()

성별
1    61
2    39
Name: w, dtype: int64

In [4]:
# Raking전 '전공'의 분포 (단위: %)
raking.groupby('전공')['w'].sum()

전공
1    51
2    49
Name: w, dtype: int64

In [5]:
# Raking 가중치를 구하기 위한 함수 정의
def sraking(w, dat, pdist):
    
    ## w: weight (numpy array)
    ## dat: sample data, should be categorical
    ## pdist: reference distribution  (numpy array)
    
    n = len(w); nu = len(pdist)
    frac = w.sum() / pdist.sum()
    
    ratio = pdist*frac  
    uniq = np.array(sorted(np.unique(dat)))
    subtotal = np.zeros(nu)
    
    for i in range(nu):
        subtotal[i] = w[np.where(dat == uniq[i])[0]].sum()
        
    fw = np.zeros(n)
    for j in range(n):
        v = np.where(uniq == dat[j])[0]
        fw[j] = w[j] * ratio[v]/subtotal[v]
    
    return fw

In [6]:
# Reference distribution
성별 = np.array([0.5, 0.5])   # 남여 성비율이 0.5:0.5 가 되게끔 가중치 계산
전공 = np.array([0.3, 0.7])   # 전공 선택 비율이 0.3:0.7이 되게끔 가중치 계산

# 가중치 변수 w만 따로 저장 (for문에서 가중치 업데이트를 위해)
w = np.array(raking['w'])

# 데이터 프레임을  numpy array로 변환 (for문에서 column을 지정하여 계산하기 위해)
raking = np.array(raking)

# Raking 가중치 계산
for i in range(10):
    w = sraking(w, raking[:,0], 성별)
    w = sraking(w, raking[:,1], 전공)


In [7]:
# In[6]에서 업데이트한 Raking가중치를 새로운 Data Frame의 w변수에 할당
raking = pd.DataFrame({'성별': raking[:,0], '전공': raking[:,1], 'w':w})
raking

,성별,전공,w
0,1,2,1.228428
1,2,1,0.714047
2,1,2,1.228428
3,1,1,0.535814
4,2,2,1.637054
...,...,...,...
95,1,2,1.228428
96,1,2,1.228428
97,2,1,0.714047
98,1,1,0.535814


In [8]:
# Raking가중치 적용 후 '성별'의 분포 (단위: %)
raking.groupby('성별')['w'].sum()

성별
1    50.0
2    50.0
Name: w, dtype: float64

In [9]:
# Raking가중치 적용 후 '전공'의 분포 (단위: %)
raking.groupby('전공')['w'].sum()

전공
1    30.0
2    70.0
Name: w, dtype: float64